In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

def set_css_in_cell_output():
    display(HTML('''
        <style>
            .jupyter-widgets {color: #d5d5d5 !important;}
            .widget-label {color: #d5d5d5 !important;}
        </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)
from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#0D0D0D"

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import time
from scipy import stats
from tqdm.notebook import tqdm
import json
import pickle

pd.options.display.max_columns = None

In [3]:
DATA_ROOT = './data/SST/'  # please contact the authors for access to the data

good_th_dict = pickle.load(open(f'{DATA_ROOT}/SST_data.pickle', 'rb'))
print(len([x for x in good_th_dict]))

60


In [4]:
dimensions = {
    'Neuroticism': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56],
    'Extraversion': [2, 7, 12, 17, 22, 27, 32, 37, 42, 47, 52, 57],
    'Openness': [3, 8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58],
    'Agreeableness': [4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59],
    'Conscientiousness': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
}

cols_of_interest = sorted([x.lower() for x in dimensions])

In [5]:
scales_df = pd.read_csv(f'{DATA_ROOT}/SST_scales.csv')
scales_df.head()

(60, 6)


,subject,agreeableness,conscientiousness,extraversion,neuroticism,openness
0,5,40.0,36.0,40.0,40.0,42.0
1,9,47.0,40.0,39.0,38.0,37.0
2,16,47.0,52.0,40.0,28.0,44.0
3,17,26.0,27.0,36.0,43.0,42.0
4,19,50.0,44.0,51.0,27.0,53.0


In [6]:
liwc_df = pd.read_csv(f'{DATA_ROOT}/SST_LIWC_2015.csv')
liwc_df['subject'] = [str(x.split('.')[0]) for x in liwc_df.Filename.values.flatten()]
liwc_df = liwc_df[liwc_df.subject.isin(scales_df.subject.values.flatten())]
liwc_df = liwc_df.drop(columns=['Filename', 'Segment'])
liwc_df = liwc_df.groupby('subject', as_index=False).mean()
print(liwc_df.shape)
liwc_df.head()

(60, 94)


,subject,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,100,3302,23.48,27.34,90.22,42.82,3302.0,10.93,94.55,62.48,20.59,13.42,9.39,0.79,0.61,2.00,0.64,7.18,4.88,14.23,10.72,8.12,8.63,2.12,21.35,5.15,4.24,1.64,0.55,2.18,4.48,2.67,1.73,0.39,0.36,0.27,9.21,0.42,0.58,1.12,1.18,15.32,3.30,1.70,2.15,3.51,1.61,5.00,2.51,0.85,1.15,0.45,1.06,0.55,0.39,0.00,0.06,6.06,2.12,1.06,1.70,1.36,0.24,4.54,14.42,1.51,15.23,2.57,7.45,5.57,1.51,1.45,0.88,0.12,0.12,0.00,0.88,0.12,0.15,0.33,0.12,0.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,101,1712,23.86,23.27,93.85,45.07,1712.0,15.60,92.87,61.62,18.46,11.16,8.82,0.12,0.41,0.88,0.93,7.30,6.13,13.84,10.34,12.62,8.18,1.29,21.32,2.45,2.92,2.98,0.58,1.34,3.15,2.10,1.05,0.41,0.06,0.29,6.60,0.76,0.41,0.53,0.93,16.36,4.73,2.51,1.17,4.91,0.29,4.32,4.38,2.04,1.64,0.70,2.92,1.81,0.18,0.06,0.99,3.74,1.46,0.41,0.99,0.53,0.35,3.04,13.55,0.76,14.43,1.81,7.01,5.72,1.69,2.39,0.76,0.00,0.00,0.00,0.35,0.00,0.12,0.06,0.18,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103,1823,9.61,11.92,80.27,37.41,1823.0,10.48,94.13,61.93,22.00,13.00,10.59,0.22,0.55,1.10,0.55,9.00,3.73,11.52,14.81,8.28,5.98,3.40,26.49,3.57,3.40,2.03,1.10,2.08,4.17,2.36,1.70,0.11,0.44,0.33,5.49,0.16,0.60,0.88,1.10,14.98,3.18,2.47,2.74,3.24,2.08,3.68,1.48,0.82,0.16,0.22,3.29,1.92,1.32,0.00,0.11,5.54,0.66,0.77,1.37,2.74,0.49,2.63,18.38,2.85,12.51,3.35,4.88,4.22,1.59,0.93,0.16,0.27,0.16,0.22,2.91,0.38,1.48,0.16,0.49,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,107,3974,25.38,28.06,67.23,38.76,3974.0,10.95,95.55,64.34,20.13,13.54,9.74,0.55,0.68,2.06,0.50,6.59,3.37,17.49,10.24,8.73,9.74,2.24,19.80,4.60,10.02,2.24,0.43,2.39,5.76,3.25,2.52,0.65,0.88,0.38,9.61,0.96,0.73,1.28,1.81,14.57,2.97,2.19,2.04,3.20,1.64,4.15,2.32,0.70,0.58,0.91,1.91,0.43,0.75,0.20,0.20,5.94,2.67,0.58,1.66,1.36,0.15,2.87,15.10,1.71,10.92,1.96,4.35,4.50,1.41,0.73,0.43,0.28,0.18,0.10,1.84,0.53,0.45,0.28,0.35,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,109,2157,11.35,21.53,62.16,4.98,2157.0,9.78,92.63,59.16,24.20,15.39,11.08,0.19,0.74,2.69,0.70,8.81,4.27,8.95,12.70,7.37,5.29,1.53,24.71,4.73,2.55,3.57,1.02,1.11,4.87,1.44,3.43,0.56,1.11,0.65,8.44,0.19,0.09,0.83,2.09,15.48,3.57,1.30,4.64,6.31,0.93,4.13,5.38,1.99,1.81,1.53,3.71,2.23,0.88,0.14,0.56,3.20,0.79,0.56,1.07,0.93,0.14,3.66,18.13,1.62,11.40,2.55,5.89,3.06,1.95,0.88,0.79,0.23,0.19,0.46,2.23,0.51,0.79,0.28,0.60,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.model_selection import KFold
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, sem

def fit_ols_models_cv(scales_df, liwc_df, outer_splits=5, random_state=42):
    """
    Perform k-fold cross-validation to compute multiple correlation (R)
    between LIWC predictors and each trait.

    Parameters:
        scales_df: DataFrame with subject and trait columns.
        liwc_df: DataFrame with subject and LIWC features.
        outer_splits: Number of outer CV folds.
    
    Returns:
        final_r: dict of mean and std_err cross-validated R per trait.
    """
    merged = pd.merge(scales_df, liwc_df, on='subject').dropna()
    assert merged.shape[0] == 60

    predictor_cols = [col for col in liwc_df.columns if col != 'subject']
    X_all = merged[predictor_cols].values
    y_all = {trait: merged[trait].values for trait in scales_df.columns if trait != 'subject'}

    outer_cv = KFold(n_splits=outer_splits, shuffle=True, random_state=random_state)
    avg_cv_r = {trait: [] for trait in y_all}

    for train_idx, test_idx in outer_cv.split(X_all):
        X_train, X_test = X_all[train_idx], X_all[test_idx]

        X_train_w_intercept = sm.add_constant(X_train)
        X_test_w_intercept = sm.add_constant(X_test)

        for trait, y in y_all.items():
            y_train, y_test = y[train_idx], y[test_idx]
            model = sm.OLS(y_train, X_train_w_intercept).fit()
            y_pred = model.predict(X_test_w_intercept)

            r, _ = pearsonr(y_test, y_pred)
            avg_cv_r[trait].append(r)

    final_r = {trait: (np.mean(r_vals), sem(r_vals)) for trait, r_vals in avg_cv_r.items()}
    return final_r

In [10]:
def pretty_print_ols_cv_results(results):
    """
    Pretty prints 5-fold CV results for OLS without PCA.

    Parameters:
        results (dict): Trait name → cross-validated R
    """
    max_len = max(len(trait) for trait in results)
    lines = []
    lines.append("5-Fold Cross-Validation Results (simple OLS)")
    lines.append("Multiple Correlation Coefficient: (trait ~ all LIWC predictors)")
    for trait in results:
        r_ci_upper = results[trait][0]+(1.96*results[trait][1])
        r_ci_lower = results[trait][0]-(1.96*results[trait][1])
        r_ci = f'({r_ci_lower:>5.3f} - {r_ci_upper:.3f})'
        lines.append(f"• {trait:<{max_len}}  R = {results[trait][0]:>5.3f}  R-SE = {results[trait][1]:>5.3f}   R-95%CI = {r_ci}")
    return "\n".join(lines)

cv_r = fit_ols_models_cv(scales_df, liwc_df)
print(pretty_print_ols_cv_results(cv_r))


5-Fold Cross-Validation Results (simple OLS)
Multiple Correlation Coefficient: (trait ~ all LIWC predictors)
• agreeableness      R = 0.135  R-SE = 0.103   R-95%CI = (-0.067 - 0.337)
• conscientiousness  R = 0.082  R-SE = 0.117   R-95%CI = (-0.147 - 0.311)
• extraversion       R = 0.052  R-SE = 0.115   R-95%CI = (-0.173 - 0.278)
• neuroticism        R = 0.436  R-SE = 0.107   R-95%CI = (0.228 - 0.645)
• openness           R = 0.282  R-SE = 0.115   R-95%CI = (0.057 - 0.507)
